# MAX17551 Circuit Calculations

This workspace intends to parametrically solve for the passive components surrounding the MAX17551 in the Typical Application Circuit.

## Circuit Diagram

The Typical Application circuit is the following:

![Typical Application Circuit](images/typical_application_circuit.png)

## Pin Descriptions

- **IN**: Switching Regulator Input. Connect a X7R 1μF ceramic capacitor from IN to GND for bypassing.

- **EN/UVLO**: Active-High, Enable/Undervoltage-Detection Input. Pull EN/UVLO to GND to disable the regulator output. Connect EN/UVLO to IN for always-on operation. Connect a resistor-divider between IN, EN/UVLO, and GND to program the input voltage at which the device is enabled and turns on.

- **RT/SYNC**: Oscillator Timing Resistor Input. Connect a resistor from RT/SYNC to GND to program the switching frequency from 100kHz to 2.2MHz. An external pulse can be applied to RT/SYNC through a coupling capacitor to synchronize the internal clock to the external pulse frequency.

- **SS**: Soft-Start Capacitor Input. Connect a capacitor from SS to GND to set the soft-start time. Leave SS unconnected for default 5.1ms internal soft-start.

- **FB**: Output Feedback Connection. Connect FB to a resistor-divider between VOUT and GND to set the output voltage.

- **VOUT**: External Bias Input for Internal Control Circuitry. Decouple to GND with a 0.22μF capacitor and connect to output capacitor positive terminal with a 22.1Ω resistor for applications with an output voltage from 3.3V to 5V. Connect to GND for output voltages < 3.3V and > 5V.

- **RESET**: Open-Drain Reset Output. Pull up RESET to an external power supply with an external resistor. RESET pulls low if FB voltage drops below 92% of its set value. RESET goes high-impedance 2ms after FB voltage rises above 95% of its set value.

- **MODE**: PFM/PWM Mode-Selection Input. Connect MODE to GND to enable the fixed-frequency PWM operation. Leave MODE unconnected for light-load PFM operation.  Note that in PFM Mode, PWM mode will automatically be enabled when the load current reaches $I_{PFM} - \frac{\Delta I}{2}$

- **GND**: Ground. Connect GND to the power ground plane. Connect all the circuit ground connections together at a single point.

- **LX**: Inductor Connection. Connect LX to the switching-side of the inductor. LX is high-impedance when the device is in shutdown.

- **EP**: Exposed Pad (TDFN Only). Connect to the GND pin to the IC.

In [625]:
# Parameter Definition - Change 

# Input Voltage
MINIMUM_INPUT_VOLTAGE = 18.0 # V
TYPICAL_INPUT_VOLTAGE = 24.0 # V
MAXIMUM_INPUT_VOLTAGE = 48.0 # V

# Output Voltage
OUTPUT_VOLTAGE = 3.3 # V

MAXIMUM_LOAD_CURRENT = 0.100 # A

INDUCTOR_RESISTANCE = 0.7 # Ω - Use 1.0 Ω at first, then fine tune with the actual inductor resistance

SOFT_START_TIME = 5.1 # ms - Leave floating for 5.1 ms

# Switching Frequency
SWITCHING_FREQUENCY = 500.0 # kHz

In [626]:
# Constant Definitions

ENABLE_THRESHOLD_RISING_MINIMUM = 1.2   # V
ENABLE_THRESHOLD_RISING_TYPICAL = 1.25  # V
ENABLE_THRESHOLD_RISING_MAXIMUM = 1.3   # V

ENABLE_THRESHOLD_FALLING_MINIMUM = 1.1  # V
ENABLE_THRESHOLD_FALLING_TYPICAL = 1.15 # V
ENABLE_THRESHOLD_FALLING_MAXIMUM = 1.2  # V

MAXIMUM_DUTY_CYCLE = 0.9
MINIMUM_CONTROLLABLE_ON_TIME = 128.0 # ns

MINIMUM_SOFT_START_TIME = 5.1 # ms

MINIMUM_INPUT_CAPACITANCE = 1.0 # uF

DECOUPLE_CAPACITANCE = 1.0 # uF
DECOUPLE_RESISTANCE = 0.0221 # kΩ
PULLUP_RESISTANCE = 100.0 # kΩ

## Input Voltage Bounds

The absolute minimum and maximum input voltages depend on the output voltage, the maximum output current, the inductor resistance, and the switching frequency.

$$V_{INMIN} = \frac{V_{OUT}+(I_{OUT}\times (R_{DCR} + 5))}{D_{MAX}}+(I_{OUT}\times 4.5)$$
$$V_{INMAX} = \frac{V_{OUT}}{t_{ONMIN}\times f_{SW}}$$

Where:

- $V_{OUT}$ = Output Voltage
- $I_{OUT}$ = Maximum Load Current
- $R_{DCR}$ = Inductor Resistance
- $f_{SW}$  = Maximum Switching Frequency
- $D_{MAX}$ = Maximum Duty Cycle (Constant)
- $t_{ONMIN}$ = Minimum Controllable Switch On-time (Constant)

Therefore, to raise the maximum input voltage, a lower switching frequency should be used.

In [627]:
# Calculate Absolute Input Voltages

ABSOLUTE_MINIMUM_INPUT_VOLTAGE = \
    (OUTPUT_VOLTAGE + (MAXIMUM_LOAD_CURRENT * (INDUCTOR_RESISTANCE + 5))) / MAXIMUM_DUTY_CYCLE \
          + MAXIMUM_LOAD_CURRENT * 4.5
print(f"Absolute Minimum Input Voltage: {ABSOLUTE_MINIMUM_INPUT_VOLTAGE:.2f} V")
ABSOLUTE_MAXIMUM_INPUT_VOLTAGE = OUTPUT_VOLTAGE / \
    (MINIMUM_CONTROLLABLE_ON_TIME * (10**-9) * SWITCHING_FREQUENCY * (10**3))
print(f"Absolute Maximum Input Voltage: {ABSOLUTE_MAXIMUM_INPUT_VOLTAGE:.2f} V")
if(MAXIMUM_INPUT_VOLTAGE > ABSOLUTE_MAXIMUM_INPUT_VOLTAGE):
    print(f"Warning: Maximum Input Voltage {MAXIMUM_INPUT_VOLTAGE} V is greater than Absolute Maximum Input Voltage")
    print("Try lowering the switching frequency")
if(MINIMUM_INPUT_VOLTAGE < ABSOLUTE_MINIMUM_INPUT_VOLTAGE):
    print(f"Warning: Minimum Input Voltage {MINIMUM_INPUT_VOLTAGE} V is less than Absolute Minimum Input Voltage")
    print("Can you reduce the load current?")
    

Absolute Minimum Input Voltage: 4.75 V
Absolute Maximum Input Voltage: 51.56 V


## Inductor Calculations

From the datasheet, we use the following equation to calculate the inductor:

$$L = \frac{18000\times V_{OUT}}{f_{SW}}$$

We can then calculate the peak-peak ripple current, $\Delta I$ for the minimum and maximum input voltages:

$$\Delta I=\frac{1000 \times V_{OUT}\times \left(1 - \frac{V_{OUT}}{V_{IN}}\right)}{f_{SW}\times L}$$

In [628]:
# Inductor Calculations

inductance = (18000 * OUTPUT_VOLTAGE) / (SWITCHING_FREQUENCY)
print(f"Inductance: {inductance:.2f} μH")

# Peak-Peak Ripple Current

minimum_ripple_current = 1000 * OUTPUT_VOLTAGE * (1 - (OUTPUT_VOLTAGE / MINIMUM_INPUT_VOLTAGE)) / (SWITCHING_FREQUENCY * inductance)
print(f"Minimum Ripple Current: ±{minimum_ripple_current:.4f} mA")
maximum_ripple_current = 1000 * OUTPUT_VOLTAGE * (1 - (OUTPUT_VOLTAGE / MAXIMUM_INPUT_VOLTAGE)) / (SWITCHING_FREQUENCY * inductance)
print(f"Maximum Ripple Current: ±{maximum_ripple_current:.4f} mA")

Inductance: 118.80 μH
Minimum Ripple Current: ±0.0454 mA
Maximum Ripple Current: ±0.0517 mA


## Output Capacitance

From the datasheet, the *minimum* output capacitance required is given by:

$$C_{OUT} = \frac{25}{V_{OUT}}$$

where $C_{OUT}$ is given in μF.

In [629]:
# Output Capacitor Selection

output_capacitance = 25/OUTPUT_VOLTAGE
print(f"Minimum Output Capacitance: {output_capacitance:.2f} μF")

Minimum Output Capacitance: 7.58 μF


## Calculating UVLO Resistor Network

A resistor divider network is used to divide the input voltage down.  The Enable pin is triggered on a maximum rising voltage of 1.3 V.  Therefore to ensure the device is enabled for the entire desired input range, the minimum input voltage is used to calculate this resistance.

![UVLO Resistor Network](images/uvlo_network.png)

In [630]:
# UVLO Resistor Calculations

# Calculate the UVLO resistors for minimum input voltage and maximum enable threshold
uvlo_resistor_ratio = ENABLE_THRESHOLD_RISING_MAXIMUM / \
    (MINIMUM_INPUT_VOLTAGE - ENABLE_THRESHOLD_RISING_MAXIMUM)
print("UVLO Resistor Ratio: {:.4f}".format(uvlo_resistor_ratio))
# Calculate Resistors
uvlo_resistor_1 = 1000 # Assume 1 MΩ
uvlo_resistor_2 = uvlo_resistor_ratio * uvlo_resistor_1
print(f"Assume UVLO Resistor 1: {uvlo_resistor_1} kΩ")
print(f"Calculate UVLO Resistor 2: {uvlo_resistor_2:.2f} kΩ")


UVLO Resistor Ratio: 0.0778
Assume UVLO Resistor 1: 1000 kΩ
Calculate UVLO Resistor 2: 77.84 kΩ


## Calculating Output Voltage Divider Network

Similarly to the UVLO configuration, the feedback pin uses a resistor divider to set the desired output voltage.  From the datasheet, the following equation is given:

$$R_1 = R_2\times \left[\frac{V_{OUT}}{0.8}-1\right]$$

![Output Voltage Divider Resistor Network](images/output_voltage_network.png)

In [631]:
# Output Voltage Divider Calculations

# Calculate the output voltage divider resistor ratio
output_divider_ratio = OUTPUT_VOLTAGE/0.8 - 1
print("Output Divider Ratio: {:.4f}".format(output_divider_ratio))

# Calculate Resistors
output_resistor_2 = 47 # Assume 47 kΩ
output_resistor_1 = output_divider_ratio * output_resistor_2
print(f"Assume Output Resistor 2: {output_resistor_2} kΩ")
print(f"Calculate Output Resistor 1: {output_resistor_1:.2f} kΩ")



Output Divider Ratio: 3.1250
Assume Output Resistor 2: 47 kΩ
Calculate Output Resistor 1: 146.87 kΩ


## Soft Start Capacitor

If no capacitor is connected to the SS pin, and it is instead left floating, there is an internal soft start delay of 5.1 ms.  However, to adjust the soft-start time, and external capacitor can be added.

If a capacitor is added, the minimum soft-start time is given by:

$$t_{SS}>0.05\times C_{OUT}\times V_{OUT}$$

where $t_{SS}$ is given in ms, and $C_{OUT}$ in μF.

Given the soft-start time is greater than the minimum, the soft-start capacitor can be calculated with:

$$C_{SS}=6.25\times t_{SS}$$

***Note*** $C_{SS}$ is in nF!

In [632]:
# Soft Start Capacitor Calculations
if(SOFT_START_TIME != MINIMUM_SOFT_START_TIME):
    soft_start_capacitance = 6.25 * SOFT_START_TIME
    print("Soft Start Capacitance: {:.2f} nF".format(soft_start_capacitance))
else :
    print("Soft Start Capacitance: Floating")
    soft_start_capacitance = 0

# Calculate minimum soft start time for the output capacitance and voltage
minimum_soft_start_time = 0.05 * output_capacitance * OUTPUT_VOLTAGE
print("Minimum Soft Start Time: {:.2f} ms".format(minimum_soft_start_time))

if(SOFT_START_TIME < minimum_soft_start_time):
    print("Warning: Soft Start Time is less than minimum soft start time")
    print("Increase the soft start time")

Soft Start Capacitance: Floating
Minimum Soft Start Time: 1.25 ms


## Sync/RT Resistor

The switching frequency can be set by adjusting the resistor $R_T$:

$$R_T = \frac{42000}{f_{SW}}$$

***Note***: The switching frequency must be between 100kHz and 2.2Mhz, and the frequency ranges of 130kHz to 160kHz, and 230 kHz to 280 kHz are not allowed.

In [633]:
# Sync Resistor Calculations

# Calculate the RT/Sync Resistor for the desired switching frequency
RT = 42000.0 / SWITCHING_FREQUENCY
print("RT/Sync Resistor for Desired Switching Frequency: {:.3f} kΩ".format(RT))

# Check that the switching frequency is within the recommended range
if SWITCHING_FREQUENCY < 100.0 or SWITCHING_FREQUENCY > 2200.0:
    print(f"WARNING: Switching Frequency {SWITCHING_FREQUENCY} kHz is outside of recommended range 100-2200 kHz")
# Check that the switching frequency is not within the blocked ranges
if SWITCHING_FREQUENCY > 130.0 and SWITCHING_FREQUENCY < 160.0:
    print(f"WARNING: Switching Frequency {SWITCHING_FREQUENCY} kHz is within blocked range 130-160 kHz")
if SWITCHING_FREQUENCY > 230.0 and SWITCHING_FREQUENCY < 280.0:
    print(f"WARNING: Switching Frequency {SWITCHING_FREQUENCY} kHz is within blocked range 230-280 kHz")


RT/Sync Resistor for Desired Switching Frequency: 84.000 kΩ


## Conclusion

Using the values calculated throughout this workbook, the suggested components are the following:

![Typical Application Circuit](images/typical_application_circuit.png)

***Note*** that the capacitor connected to SS should be $C_{SS}$.

In [634]:
print(f"C_IN >= {MINIMUM_INPUT_CAPACITANCE:.2f} μF")
print(f"R1 = {uvlo_resistor_1:.2f} kΩ")
print(f"R2 = {uvlo_resistor_2:.2f} kΩ")
print(f"C_SS = {soft_start_capacitance:.2f} nF")
print(f"R3 = {RT:.3f} kΩ")
print(f"L1 = {inductance:.2f} μH")
print(f"C_OUT = {output_capacitance:.2f} μF")
if(OUTPUT_VOLTAGE >= 3.3 and OUTPUT_VOLTAGE <= 5):
    print(f"C_F = {DECOUPLE_CAPACITANCE:.2f} μF")
    print(f"R7 = {DECOUPLE_RESISTANCE:.2f} kΩ")
else:
    print(f"C_F = 0.0 μF")
    print(f"R7 = NC")
    print(f"V_OUT should be tied to ground")
print(f"R4 = {output_resistor_1:.2f} kΩ")
print(f"R5 = {output_resistor_2:.2f} kΩ")
print(f"R6 = {PULLUP_RESISTANCE} kΩ")


C_IN >= 1.00 μF
R1 = 1000.00 kΩ
R2 = 77.84 kΩ
C_SS = 0.00 nF
R3 = 84.000 kΩ
L1 = 118.80 μH
C_OUT = 7.58 μF
C_F = 1.00 μF
R7 = 0.02 kΩ
R4 = 146.87 kΩ
R5 = 47.00 kΩ
R6 = 100.0 kΩ


## Example

This is an example of a 3.3 V, 50mA Regulator that takes advantage of PFM for light loads with a 300 kHz switching frequency:

![3V3 Example Circuit](images\3V3_example.png)